# Coastal protection layer preparation

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

In [2]:
df = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/Mangrove_Coastal_Protection_Stats_20230426.csv', skiprows=1)
df.head()

,Name,Area,Population,Stock,Area.1,Population.1,Stock.1,Area.2,Population.2,Stock.2
0,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,NaN,3128.645656,2.381327e+06,0.000000,7241.891811,3.939056e+06,0.000000,14039.010650,12073501.76
2,Anguilla,0.615599,0.000000,5.042979e+06,0.922914,0.000000,2.926016e+06,2.781378,0.000000,7497182.30
3,Antigua and Barbuda,2.375643,312.024456,1.553655e+07,2.835863,1208.306314,3.053302e+07,9.206714,1144.118241,70476111.40
4,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
period = ['annual', '25_year', '100_year']
impact = ['area', 'population', 'stock']
cols_names = []

for p, i in product(period, impact):
    cols_names.append(f'{p}_{i}')
cols_names

['annual_area',
 'annual_population',
 'annual_stock',
 '25_year_area',
 '25_year_population',
 '25_year_stock',
 '100_year_area',
 '100_year_population',
 '100_year_stock']

In [5]:
df.columns = ['country'] + cols_names
df.head(3)

,country,annual_area,annual_population,annual_stock,25_year_area,25_year_population,25_year_stock,100_year_area,100_year_population,100_year_stock
0,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,NaN,3128.645656,2381327.251,0.000000,7241.891811,3939055.710,0.000000,14039.01065,12073501.76
2,Anguilla,0.615599,0.000000,5042979.030,0.922914,0.000000,2926016.389,2.781378,0.00000,7497182.30


## Prepare map layer

In [6]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [7]:
df[df['country'].isin(locations['name']) == False]['country'].unique()

array(['Congo', "Cote d'Ivoire", 'Curacao', 'Mexico',
       'Sao Tome and Principe', 'Worldwide'], dtype=object)

In [20]:
pattern = 'Tom'
locations[locations['name'].str.contains(pattern, case=False)][['name', 'iso']]

,name,iso
1472,São Tomé and Príncipe,STP


In [21]:
df.loc[df['country'] == 'Congo', 'country'] = 'Republic of the Congo'
df.loc[df['country'] == "Cote d'Ivoire", 'country'] = "Côte d'Ivoire"
df.loc[df['country'] == 'Curacao', 'country'] = 'Curaçao'
df.loc[df['country'] == 'Mexico', 'country'] = 'México'
df.loc[df['country'] == 'Sao Tome and Principe', 'country'] = 'São Tomé and Príncipe'

In [22]:
df[df['country'].isin(locations['name']) == False]['country'].unique()

array(['Worldwide'], dtype=object)

In [26]:
df_merged = locations.merge(df, left_on='name', right_on='country', how='left')
final_cols = ['name', 'iso', 'geometry'] + cols_names
df_merged = df_merged[final_cols]
df_merged.head()

,name,iso,geometry,annual_area,annual_population,annual_stock,25_year_area,25_year_population,25_year_stock,100_year_area,100_year_population,100_year_stock
0,Qatar,QAT,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24...",0.009636,0.000,2.363751e+06,0.285264,0.000,6.429735e+05,0.281659,0.000,8.330734e+05
1,Mayotte,MYT,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Vietnam,VNM,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ...",3116.312773,7023871.451,6.445398e+09,7693.250381,5526401.493,4.418621e+09,9710.003384,7604768.261,7.473397e+09
3,Grenada,GRD,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ...",0.018471,0.000,3.315313e+04,0.016780,0.000,2.648764e+04,0.105622,0.000,0.000000e+00
4,India,IND,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3...",1630.839037,2868567.159,7.835796e+09,3512.755950,5612753.274,1.289593e+10,4773.458384,8437411.951,1.926960e+10


In [28]:
coastal_protection_gdf = gpd.GeoDataFrame(df_merged, geometry='geometry')
coastal_protection_gdf.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [29]:
coastal_protection_gdf.to_file('../../../../data/coastal_protection_map.geojson', driver="GeoJSON") 

In [30]:
in_file = '../../../../data/coastal_protection_map.geojson'

out_file = '../../../../data/coastal_protection_map.mbtiles'

!tippecanoe -zg -f -P -o $out_file --extend-zooms-if-still-dropping $in_file


For layer 0, using name "coastal_protection_map"
../../../../data/coastal_protection_map.geojson:13: Found ] at top level
../../../../data/coastal_protection_map.geojson:19: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
122 features, 8678500 bytes of geometry, 4871 bytes of separate metadata, 12950 bytes of string pool
Choosing a maxzoom of -z0 for features about 3191542 feet (972782 meters) apart
Choosing a maxzoom of -z9 for resolution of about 749 feet (228 meters) within features
  99.9%  9/396/240  
